In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

In [3]:
train.head()

,grade_A_Component_1,grade_A_Component_2,max_luminosity,thickness,xmin,xmax,ymin,ymax,pixel_area,log_area,x_component_1,x_component_2,x_component_3,x_component_4,x_component_5,class
0,0,1,150,36,144,172,947225,947332,439,439.099273,0,0,1,0,0,1
1,1,0,134,55,1144,1152,2379058,2379624,329,329.205616,1,0,0,0,0,1
2,1,0,135,65,950,974,1038442,1036754,300,300.120598,0,0,0,0,0,2
3,0,1,123,35,41,220,1705580,1705604,6803,6803.778622,0,0,1,0,0,1
4,1,0,138,69,462,466,1088124,1086579,251,251.401943,0,0,0,0,0,2


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1358 entries, 0 to 1357
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   grade_A_Component_1  1358 non-null   int64  
 1   grade_A_Component_2  1358 non-null   int64  
 2   max_luminosity       1358 non-null   int64  
 3   thickness            1358 non-null   int64  
 4   xmin                 1358 non-null   int64  
 5   xmax                 1358 non-null   int64  
 6   ymin                 1358 non-null   int64  
 7   ymax                 1358 non-null   int64  
 8   pixel_area           1358 non-null   int64  
 9   log_area             1358 non-null   float64
 10  x_component_1        1358 non-null   int64  
 11  x_component_2        1358 non-null   int64  
 12  x_component_3        1358 non-null   int64  
 13  x_component_4        1358 non-null   int64  
 14  x_component_5        1358 non-null   int64  
 15  class                1358 non-null   i

In [5]:
train.columns

Index(['grade_A_Component_1', 'grade_A_Component_2', 'max_luminosity',
       'thickness', 'xmin', 'xmax', 'ymin', 'ymax', 'pixel_area', 'log_area',
       'x_component_1', 'x_component_2', 'x_component_3', 'x_component_4',
       'x_component_5', 'class'],
      dtype='object')

In [6]:
X = train.iloc[:,:-1]
y = train.iloc[:,-1]

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import  GridSearchCV, train_test_split
from sklearn.metrics import log_loss

In [8]:
sc = StandardScaler()

In [9]:
X = sc.fit_transform(X)
test = sc.fit_transform(test)

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2,random_state=101)

In [116]:
def model_pred(model):
    model.fit(X_train,y_train)
    model_pred = model.predict_proba(X_val)
    model_score = log_loss(y_val,model_pred)
    return model_score, model_pred

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from ngboost import NGBClassifier

/home/yash/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/yash/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/home/yash/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [16]:
def griddy_boi(model,param_grid):
    grid_search = GridSearchCV(model,param_grid,cv=3,n_jobs = -1, scoring='neg_log_loss')
    grid_search.fit(X_train,y_train)
    cvres = grid_search.cv_results_
    for mean_score,params in zip(cvres['mean_test_score'],cvres['params']):
        print((-mean_score),params)
    

In [23]:
def predis(model):
    model.fit(X,y)
    model_preds = model.predict_proba(test)
    return model_preds

## Decision Trees

In [14]:
dc = DecisionTreeClassifier()

In [18]:
dc_param_grid = [
 
    {'max_depth':[10,20,30],'min_samples_split':[100,150,200,250,300,400]}
]

In [19]:
griddy_boi(dc,dc_param_grid)

0.4547623591290113 {'max_depth': 10, 'min_samples_split': 100}
0.34312235421651405 {'max_depth': 10, 'min_samples_split': 150}
0.33098956912951927 {'max_depth': 10, 'min_samples_split': 200}
0.3415657521283273 {'max_depth': 10, 'min_samples_split': 250}
0.3415657521283273 {'max_depth': 10, 'min_samples_split': 300}
0.35712818556606324 {'max_depth': 10, 'min_samples_split': 400}
0.45693119500566465 {'max_depth': 20, 'min_samples_split': 100}
0.34312235421651405 {'max_depth': 20, 'min_samples_split': 150}
0.33098956912951927 {'max_depth': 20, 'min_samples_split': 200}
0.3415657521283273 {'max_depth': 20, 'min_samples_split': 250}
0.3415657521283273 {'max_depth': 20, 'min_samples_split': 300}
0.35712818556606324 {'max_depth': 20, 'min_samples_split': 400}
0.457099389975633 {'max_depth': 30, 'min_samples_split': 100}
0.34312235421651405 {'max_depth': 30, 'min_samples_split': 150}
0.33098956912951927 {'max_depth': 30, 'min_samples_split': 200}
0.3415657521283273 {'max_depth': 30, 'min_sampl

In [49]:
dtc = DecisionTreeClassifier(min_samples_split=150,max_depth=20)

In [117]:
dtc_score,dtc_val_pred = model_pred(dtc)

In [51]:
dtc_score

0.2980457820898426

In [52]:
dtc_predis = predis(dtc)

In [53]:
dtc_predis

array([[1.        , 0.        ],
       [0.08064516, 0.91935484],
       [1.        , 0.        ],
       ...,
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ]])

## Random Forest

In [54]:
rf = RandomForestClassifier()

In [55]:
rf_param_grid = [
    {'max_depth':[10,20,50,100]},
    {'max_depth':[10,20,50,100],'min_samples_split':[10,100,200]},
    {'max_depth':[10,20,50,100],'min_samples_split':[10,100,200],'min_samples_leaf':[2,10,50,100]},
    {'max_depth':[10,20,50,100],'min_samples_split':[10,100,200],'min_samples_leaf':[2,10,50,100],'n_estimators':[50,100,200,500]},
    {'max_depth':[10,20,50,100],'min_samples_split':[10,100,200],'min_samples_leaf':[2,10,50,100],'n_estimators':[50,100,200,500],'bootstrap':[False]},

]

In [56]:
griddy_boi(rf,rf_param_grid)

0.36282928532998454 {'max_depth': 10}
0.35770618606226506 {'max_depth': 20}
0.3586216515622906 {'max_depth': 50}
0.3577536867349656 {'max_depth': 100}
0.37937084091234624 {'max_depth': 10, 'min_samples_split': 10}
0.4196886270515419 {'max_depth': 10, 'min_samples_split': 100}
0.4684292364096099 {'max_depth': 10, 'min_samples_split': 200}
0.3669584697004578 {'max_depth': 20, 'min_samples_split': 10}
0.42109517279750536 {'max_depth': 20, 'min_samples_split': 100}
0.47175751437777097 {'max_depth': 20, 'min_samples_split': 200}
0.3657889551626388 {'max_depth': 50, 'min_samples_split': 10}
0.4283137543076366 {'max_depth': 50, 'min_samples_split': 100}
0.4753548809357735 {'max_depth': 50, 'min_samples_split': 200}
0.36817024744448995 {'max_depth': 100, 'min_samples_split': 10}
0.4224301981269702 {'max_depth': 100, 'min_samples_split': 100}
0.4716876921356598 {'max_depth': 100, 'min_samples_split': 200}
0.37671931424512106 {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10}
0.42

In [67]:
rfc = RandomForestClassifier(bootstrap=False,n_estimators=200,max_depth=20,min_samples_split=10,min_samples_leaf=2)

In [118]:
rfc_score,rfc_val_pred = model_pred(rfc)
rfc_score

0.2897179763700222

In [74]:
rfc_predis = predis(rfc)

In [75]:
rfc_predis

array([[0.89662328, 0.10337672],
       [0.20751853, 0.79248147],
       [0.88765675, 0.11234325],
       ...,
       [0.99270833, 0.00729167],
       [0.62268606, 0.37731394],
       [0.81506944, 0.18493056]])

## Extra Trees 

In [76]:
etc = ExtraTreesClassifier(bootstrap=False,n_estimators=200,max_depth=20,min_samples_split=10,min_samples_leaf=2)

In [119]:
etc_score,etc_val_pred = model_pred(etc)
etc_score

0.3128000577968881

In [78]:
etc_predis = predis(etc)
etc_predis

array([[0.9721884 , 0.0278116 ],
       [0.20025095, 0.79974905],
       [0.92237262, 0.07762738],
       ...,
       [0.98316434, 0.01683566],
       [0.81219816, 0.18780184],
       [0.83964151, 0.16035849]])

## XGBoost

In [113]:
xgbc = XGBClassifier(learning_rate=0.1,n_estimators=100)

In [120]:
xgbc_score,xgbc_val_pred = model_pred(etc)
xgbc_score

0.3161904952653854

In [115]:
xgbc_predis = predis(xgbc)
xgbc_predis

array([[0.98822486, 0.01177513],
       [0.13545352, 0.8645465 ],
       [0.99132305, 0.00867697],
       ...,
       [0.99515295, 0.00484702],
       [0.9797    , 0.02029999],
       [0.95567816, 0.04432185]], dtype=float32)

## LightGBM

In [93]:
lgbm = LGBMClassifier()

In [94]:
lgbm_param_grid = [
    {'learning_rate':[0.001,0.01,0.1,1,10]},
    {'learning_rate':[0.001,0.01,0.1,1,10],'n_estimators':[50,100,200,500]},
]

In [95]:
griddy_boi(lgbm,lgbm_param_grid)

0.5958526603936507 {'learning_rate': 0.001}
0.39646503353094503 {'learning_rate': 0.01}
0.3950887791007587 {'learning_rate': 0.1}
1.1161549840540064 {'learning_rate': 1}
15.898357799991345 {'learning_rate': 10}
0.6179057001941022 {'learning_rate': 0.001, 'n_estimators': 50}
0.5958526603936507 {'learning_rate': 0.001, 'n_estimators': 100}
0.557322716573081 {'learning_rate': 0.001, 'n_estimators': 200}
0.47320588478738385 {'learning_rate': 0.001, 'n_estimators': 500}
0.4729549019902251 {'learning_rate': 0.01, 'n_estimators': 50}
0.39646503353094503 {'learning_rate': 0.01, 'n_estimators': 100}
0.3428959445584601 {'learning_rate': 0.01, 'n_estimators': 200}
0.34203502165311045 {'learning_rate': 0.01, 'n_estimators': 500}
0.3401177660105865 {'learning_rate': 0.1, 'n_estimators': 50}
0.3950887791007587 {'learning_rate': 0.1, 'n_estimators': 100}
0.5551192935290332 {'learning_rate': 0.1, 'n_estimators': 200}
0.9412216903206015 {'learning_rate': 0.1, 'n_estimators': 500}
1.0544880981294857 {'l

In [100]:
lgbmc = LGBMClassifier(learning_rate=0.1,n_estimators=100)

In [121]:
lgbmc_score,lgbmc_val_pred = model_pred(lgbmc)
lgbmc_score

0.2945967207314328

In [103]:
lgbmc_predis = predis(lgbmc)
lgbmc_predis

array([[9.99721270e-01, 2.78730136e-04],
       [7.04919801e-02, 9.29508020e-01],
       [9.98543548e-01, 1.45645195e-03],
       ...,
       [9.99284210e-01, 7.15789839e-04],
       [9.98955035e-01, 1.04496464e-03],
       [9.95352749e-01, 4.64725101e-03]])

In [122]:
boosted_predis = 0.1*dtc_predis+0.15*rfc_predis+0.15*etc_predis+0.3*xgbc_predis+0.3*lgbmc_predis

In [123]:
boosted_predis

array([[0.97670562, 0.02329441],
       [0.13101359, 0.86898642],
       [0.9684644 , 0.03153562],
       ...,
       [0.99471206, 0.00528794],
       [0.90882917, 0.09117085],
       [0.93351594, 0.06648409]])

In [124]:
boost_predis = pd.DataFrame(boosted_predis,columns=['1','2'])

filename = 'submit_boosted.xlsx'
boost_predis.to_excel(filename,index=False)